In this notebook we will make an n-dimensional mass spring damper solver by hand (i.e. without using premade julia functions)

In [2]:
#Imports needed for this project
using Plots
using SparseArrays


In [3]:
#Parameters 
n_masses = 5
k_lst = 5*ones(1,n_masses+1)
c_lst = 0.2*ones(1,n_masses+1)  
f_lst = zeros(1,n_masses) #N
mass_vec = 0.1* ones(n_masses) #kg

tStart = 0
tEnd = 20
dt = 0.01


0.01

In [4]:
#IC's
u0_vec = 4*ones(n_masses)
v0_vec = zeros(n_masses) 

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [5]:
#Creating System Matrix A
function ConstructSystemMatrixA(n_masses,k_lst,c_lst,f_lst)
    
    # Making Diagnals for Sparse Matrix
    # Main diag K
    k_main_diag = zeros(n_masses)
    for i in 1:n_masses
        k_main_diag[i] = -k_lst[i] - k_lst[i+1] 
    end
    
    # Upper diag K
    k_upper_diag = zeros(n_masses -1)
    for i in 1:n_masses-1
        k_upper_diag[i] =  k_lst[i+1]
    end
    k_upper_diag = vcat(0,k_upper_diag)

    # Lower diag K
    k_lower_diag = zeros(n_masses -1)
    for i in 1:n_masses-1
        k_lower_diag[i] =  k_lst[i]
    end

    # Main diag C
    c_main_diag = zeros(n_masses)
    for i in 1:n_masses
        c_main_diag[i] = -c_lst[i] - c_lst[i+1] 
    end
    
    # Upper diag C
    c_upper_diag = zeros(n_masses -1)
    for i in 1:n_masses-1
        c_upper_diag[i] =  c_lst[i+1]
    end

    # Lower diag C
    c_lower_diag = zeros(n_masses -1)
    for i in 1:n_masses-1
        c_lower_diag[i] =  c_lst[i]
    end

    # 1's diagonal
    ones_diag = ones(n_masses)

    # Real main diagonal
    final_main_diag = vcat(zeros(n_masses),c_main_diag)
    final_upper_diag = vcat(zeros(n_masses),c_upper_diag)
    final_lower_diag = vcat(zeros(n_masses),c_lower_diag)


    # Building the Sparse (n_masses x n_masses) Matrix from diags
    A = spdiagm(
        0 => final_main_diag,
        1 => final_upper_diag,
        -1 => final_lower_diag,
        n_masses => ones_diag,
        -n_masses => k_main_diag,
        -n_masses+1 => k_upper_diag,
        -n_masses-1 => k_lower_diag
    )

    # TODO: Benchmark for n = 10,100,1000

    M = spdiagm(
        0 => vcat(ones(n_masses),mass_vec)
    )

    return A, M 
end

A,M = ConstructSystemMatrixA(n_masses,k_lst,c_lst,f_lst)
display(Matrix(A))
display(Matrix(M))

10×10 Matrix{Float64}:
   0.0    0.0    0.0    0.0    0.0   1.0   0.0   0.0   0.0   0.0
   0.0    0.0    0.0    0.0    0.0   0.0   1.0   0.0   0.0   0.0
   0.0    0.0    0.0    0.0    0.0   0.0   0.0   1.0   0.0   0.0
   0.0    0.0    0.0    0.0    0.0   0.0   0.0   0.0   1.0   0.0
   0.0    0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   1.0
 -10.0    5.0    0.0    0.0    0.0  -0.4   0.2   0.0   0.0   0.0
   5.0  -10.0    5.0    0.0    0.0   0.2  -0.4   0.2   0.0   0.0
   0.0    5.0  -10.0    5.0    0.0   0.0   0.2  -0.4   0.2   0.0
   0.0    0.0    5.0  -10.0    5.0   0.0   0.0   0.2  -0.4   0.2
   0.0    0.0    0.0    5.0  -10.0   0.0   0.0   0.0   0.2  -0.4

10×10 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.1  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.1

In [6]:
# Define the Forward Euler function
function forward_euler(tStart, tEnd, dt, A, u0_vec, v0_vec,M)
    num_steps = Int((tEnd - tStart) / dt) + 1
    displacement_matrix = zeros(2 * length(u0_vec), num_steps)  # Store both displacement and velocity
    t_vector = tStart:dt:tEnd  # Time vector

    w = vcat(u0_vec, v0_vec)  # Initial state
    displacement_matrix[:, 1] .= w  # Store initial state


    for i in 2:num_steps
        
        b = A*w
        
        wprime = M \ b  # Compute the derivative based on system matrix A
        w_2 = w + dt * wprime  # Update state using Forward Euler
        displacement_matrix[:, i] .= w_2  # Store new state
        w = w_2  # Update w for the next iteration
    end

    return t_vector, displacement_matrix
end

# Call the Forward Euler function
t_vector, displacement_matrix = forward_euler(tStart, tEnd, dt, A, u0_vec, v0_vec,M)


(0.0:0.01:20.0, [4.0 4.0 … -0.0308567824989897 -0.02960109124972288; 4.0 4.0 … -0.05344551504628494 -0.05127059400394872; … ; 0.0 0.0 … 0.21749210423362217 0.22348690885769001; 0.0 -2.0 … 0.12556912492668162 0.12903022699013406])

In [7]:
# Plot the results
# Assuming displacement_matrix is organized as: [displacement1; velocity1; displacement2; velocity2; ...]
mass_indices = 1:n_masses  # Mass number indices
surface(t_vector, mass_indices, displacement_matrix[1:n_masses,:], xlabel="Time (s)", ylabel="Mass Number", zlabel="Displacement (m)",
        title="Displacement of Masses Over Time", color=:viridis, legend=false)

# Show the plot
display(plot!)


plot! (generic function with 4 methods)

In [ ]:
# Animation
anim = @animate for i in 1:size(displacement_matrix[1:n_masses,:], 2)
    plot(1:n_masses, displacement_matrix[1:n_masses, i], 
         xlabel="Mass Number", 
         ylabel="Displacement (m)", 
         title="Displacement of Each Mass Over Time\n t = $(round(t_vector[i], digits=2))", 
         ylim=(-5, 5),  # Adjust according to your displacement range
         label="", 
         legend=false)
end

# Save the animation
gif(anim, "mass_displacement_animation.gif", fps=500)